In [ ]:
import iar_project.utils as utils
import iar_project.features_extraction as extraction
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
# How many images ?
nb_im = 12
seg_img=utils.import_seg_results(nb_im)
sol_img=utils.import_solution()
sol_img=[sol_img]



In [ ]:
def clustering(img_test,pca_color,pca_gray,eps_color,eps_gray,min_samp_color,min_samp_gray):
    labels = np.zeros(len(img_test))
    idx_gray = []
    idx_color = []
    clustered_puzzle = []
    for t,tile in enumerate(img_test):
        if (tile[40:80,40:80,0] == tile[40:80,40:80,1]).all():
            idx_gray.append(t)
        else:
            idx_color.append(t)
    if idx_color:
        seg_color = [[img_test[i][10:-11,10:-11] for i in idx_color]]
        features_color = extraction.get_features(seg_color,2)
        exp = 0
        while np.sum(exp) < 0.8:
            pca_color = pca_color+1
            ft_2D_color, exp = extraction.ft_PCA(features_color[0],pca_color)
        labels_color = extraction.ft_DBSCAN(ft_2D_color,eps_color,min_samp_color)
        labels[idx_color] = labels_color
        print(exp)

    if idx_gray:
        seg_gray = [[img_test[i][20:-21,20:-21] for i in idx_gray]]
        features_gray = extraction.get_features(seg_gray,1)
        exp = 0
        while np.sum(exp) < 0.8:
            pca_gray = pca_gray+1
            ft_2D_gray, exp = extraction.ft_PCA(features_gray[0],pca_gray)
        print(exp)
        labels_gray = extraction.ft_DBSCAN(ft_2D_gray,eps_gray,min_samp_gray)
        if idx_color:
            outlier = (labels_gray == -1)
            labels[idx_gray] = outlier*labels_gray
            normal = (labels_gray != -1)
            labels[idx_gray] = normal*(labels_gray + max(np.unique(labels_color))+1)
        else:
            labels[idx_gray] = labels_gray
        
        
    unique_lab = np.unique(labels)

    for l,lab in enumerate(unique_lab):
        idx = np.argwhere(labels==lab)
        puzzle = []
        for i,id in enumerate(idx):
            puzzle.append(img_test[id[0]])
        clustered_puzzle.append(puzzle)
    return clustered_puzzle, unique_lab


def plot_clustering_results(clu_img,sizefig,lon):
    for imID,img in enumerate(clu_img):
        fig, axs = plt.subplots(len(img), lon, figsize=sizefig)
        fig.suptitle(f"Image_{imID}")
        for p,puzzle in enumerate(img):
            for i,tile in enumerate(puzzle):
                if len(img) == 1:
                    axs[i].imshow(tile)
                else:
                    axs[p,i].imshow(tile)
    

In [ ]:
clu_img = []
clu_labels = []
for imID,img_test in enumerate(seg_img):
    clu, clu_lab = clustering(img_test, pca_color=0,pca_gray=0,eps_color=1.5,eps_gray=0.5,min_samp_color=4,min_samp_gray=6)
    clu_img.append(clu)
    clu_labels.append(clu_lab)

plot_clustering_results(clu_img,sizefig = (25,8),lon=25)

In [ ]:
clu_img = []
clu_labels = []
for imID,img_test in enumerate(sol_img):
    clu, clu_lab = clustering(img_test, pca_color=0,pca_gray=0,eps_color=1.5,eps_gray=0.5,min_samp_color=1,min_samp_gray=1)
    clu_img.append(clu)
    clu_labels.append(clu_lab)

plot_clustering_results(clu_img,sizefig=(10,20),lon=7)
